# Getting metrics for the first model

Let's try to update my report with the metrics for the first model.

As per the tests conducted on the first notebook, regarding the datasets (to find which datasets can be run in a reasonable time), we are going to use these default datasets:

* `birds`
* `emotions`
* `scene`

The **baseline** will be the regular Binary Relevance. Then we will compare those to the Basic Stacking approach. Finally, we will run the Stacking With F-Test, with `alpha=0.5`. All other parameters, of the other models, will be the default ones, using the `SVC` as the base classifier.

The metrics we will use, as per what was defined in my report, are the hamming loss and the f1 score. We will use the `EvaluationPipeline` class to run the experiments.

## Setup

In [7]:
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.dataset import load_dataset, available_data_sets
from sklearn.svm import SVC
from skmultilearn.base.problem_transformation import ProblemTransformationBase
from typing import List, Optional, Any, Tuple, Dict
import numpy as np
import sklearn.metrics as metrics
import json
import pandas as pd
from sklearn.feature_selection import f_classif
from evaluation import EvaluationPipeline


## Models

In [5]:
# TODO: move this to an actual python file

class BasicStacking(ProblemTransformationBase):
    first_layer_classifiers: BinaryRelevance
    second_layer_classifiers: BinaryRelevance

    def __init__(self, classifier: Any = None, require_dense: Optional[List[bool]] = None):
        super(BasicStacking, self).__init__(classifier, require_dense)

        self.first_layer_classifiers = BinaryRelevance(
            classifier=SVC(),
            require_dense=[False, True]
        )

        self.second_layer_classifiers = BinaryRelevance(
            classifier=SVC(),
            require_dense=[False, True]
        )
    
    def fit(self, X: Any, y: Any):
        self.first_layer_classifiers.fit(X, y)

        first_layer_predictions = self.first_layer_classifiers.predict(X)
        X_expanded = np.hstack([X.todense(), first_layer_predictions.todense()])

        self.second_layer_classifiers.fit(X_expanded, y)
    
    def predict(self, X: Any):
        first_layer_predictions = self.first_layer_classifiers.predict(X)
        X_expanded = np.hstack([X.todense(), first_layer_predictions.todense()])
        return self.second_layer_classifiers.predict(X_expanded)


In [6]:
# TODO move this to an actual python file

class StackingWithFTests(ProblemTransformationBase):
    alpha: float
    use_first_layer_to_calculate_correlations: bool
    
    first_layer_classifiers: BinaryRelevance
    second_layer_classifiers: List[Any] # TODO should be any generic type of classifier
    labels_count: int

    def __init__(
        self,
        alpha: float = 0.5,
        use_first_layer_to_calculate_correlations: bool = False,
        classifier: Any = None,
        require_dense: Optional[List[bool]] = None
    ):
        super(StackingWithFTests, self).__init__(classifier, require_dense)

        if alpha < 0.0 or alpha > 1.0:
            raise Exception("alpha must be >= 0.0 and <= 1.0")

        self.alpha = alpha
        self.use_first_layer_to_calculate_correlations = use_first_layer_to_calculate_correlations
        
        self.first_layer_classifiers = BinaryRelevance(
            classifier=SVC(),
            require_dense=[False, True]
        )
        # TODO: allow for any base model (base classifier) to be used
        # right now I am forcing the use of SVC

        self.second_layer_classifiers = []
        self.correlated_labels_map = pd.DataFrame()
        self.labels_count = 0


    def fit(self, X: Any, y: Any):
        self.labels_count = y.shape[1]

        self.first_layer_classifiers.fit(X, y)
        
        label_classifications = y
        if self.use_first_layer_to_calculate_correlations:
            label_classifications = self.first_layer_classifiers.predict(X)

        f_tested_label_pairs = self.calculate_f_test_for_all_label_pairs(label_classifications)
        self.correlated_labels_map = self.get_map_of_correlated_labels(f_tested_label_pairs)

        for i in range(self.labels_count):
            mask = self.correlated_labels_map["for_label"] == i
            split_df = self.correlated_labels_map[mask].reset_index(drop=True)
            labels_to_expand = split_df["expand_this_label"].to_list()

            additional_input = label_classifications.todense()[:, labels_to_expand]
            
            X_expanded = np.hstack([X.todense(), additional_input])
            X_expanded = np.asarray(X_expanded)

            y_label_specific = y.todense()[:, i]
            y_label_specific = self.convert_matrix_to_vector(y_label_specific)

            meta_classifier = SVC()
            meta_classifier.fit(X_expanded, y_label_specific)

            self.second_layer_classifiers.append(meta_classifier)
            print(f"finished training meta classifier for label {i}")
    
    def calculate_f_test_for_all_label_pairs(self, label_classifications: Any) -> List[Dict[str, Any]]:
        results = []

        for i in range(0, self.labels_count):
            for j in range(0, self.labels_count):
                if i == j:
                    continue

                X = label_classifications.todense()[:, i]
                base_label = self.convert_matrix_to_array(X)

                y = label_classifications.todense()[:, j]
                against_label = self.convert_matrix_to_vector(y)

                f_test_result = f_classif(base_label, against_label)[0]

                results.append({
                    "label_being_tested": i,
                    "against_label": j,
                    "f_test_result": float(f_test_result)
                })
        
        return results
    
    def convert_matrix_to_array(self, matrix: Any):
        return np.asarray(matrix).reshape(-1, 1)

    def convert_matrix_to_vector(self, matrix: Any):
        return np.asarray(matrix).reshape(-1)
    
    def get_map_of_correlated_labels(self, f_test_results: List[Dict[str, Any]]) -> pd.DataFrame:
        temp_df = pd.DataFrame(f_test_results)
        
        sorted_temp_df = temp_df.sort_values(
            by=["label_being_tested", "f_test_result"],
            ascending=[True, False])
        # ordering in descending order by the F-test result,
        # following what the main article describes

        selected_features = []

        for i in range(0, self.labels_count):
            mask = sorted_temp_df["label_being_tested"] == i
            split_df = sorted_temp_df[mask].reset_index(drop=True)

            big_f = split_df["f_test_result"].sum()
            max_cum_f = self.alpha * big_f

            cum_f = 0
            for _, row in split_df.iterrows():
                cum_f += row["f_test_result"]
                if cum_f > max_cum_f:
                    break

                selected_features.append({
                    "for_label": i,
                    "expand_this_label": int(row["against_label"]),
                    "f_test_result": float(row["f_test_result"]),
                })
        
        cols = ["for_label", "expand_this_label", "f_test_result"]
        return pd.DataFrame(selected_features, columns=cols)
    
    def predict(self, X: Any) -> np.ndarray[Any,Any]:
        if self.correlated_labels_map.columns.size == 0:
            raise Exception("model was not trained yet")

        predictions = self.first_layer_classifiers.predict(X)
        local_labels_count = predictions.shape[1]

        second_layer_predictions = []

        for i in range(local_labels_count):
            mask = self.correlated_labels_map["for_label"] == i
            split_df = self.correlated_labels_map[mask].reset_index(drop=True)
            labels_to_expand = split_df["expand_this_label"].to_list()

            additional_input = predictions.todense()[:, labels_to_expand]

            X_expanded = np.hstack([X.todense(), additional_input])
            X_expanded = np.asarray(X_expanded)

            temp_preds = self.second_layer_classifiers[i].predict(X_expanded)
            second_layer_predictions.append(temp_preds)

        reshaped_array = np.asarray(second_layer_predictions).T
        return reshaped_array


## Datasets

In [11]:
desired_datasets = ["scene", "emotions", "birds"]

datasets = {}
for dataset_name in desired_datasets:
    print(f"getting dataset `{dataset_name}`")
    
    full_dataset = load_dataset(dataset_name, "undivided")
    X, y, _, _ = full_dataset

    datasets[dataset_name] = {
        "X": X,
        "y": y,
        "rows": X.shape[0],
        "labels_count": y.shape[1]
    }


for name, info in datasets.items():
    print("===")
    print(f"information for dataset `{name}`")
    print(f"rows: {info['rows']}, labels: {info['labels_count']}")


getting dataset `scene`
scene:undivided - exists, not redownloading
getting dataset `emotions`
emotions:undivided - exists, not redownloading
getting dataset `birds`
birds:undivided - exists, not redownloading
===
information for dataset `scene`
rows: 2407, labels: 6
===
information for dataset `emotions`
rows: 593, labels: 6
===
information for dataset `birds`
rows: 645, labels: 19


## Evaluations

In [13]:
baseline_binary_relevance_model = BinaryRelevance(
    classifier=SVC(),
    require_dense=[False, True]
)

basic_stacking_model = BasicStacking()
stacking_with_f_tests_model = StackingWithFTests(alpha=0.5)

models = {
    "baseline_binary_relevance_model": baseline_binary_relevance_model,
    "basic_stacking_model": basic_stacking_model,
    "stacking_with_f_tests_model": stacking_with_f_tests_model
}



In [ ]:
evaluation_results = {}


In [19]:
for model_name, model in models.items():
    print(f"# running model `{model_name}`")

    evaluation_results[model_name] = {}

    n_folds = 5
    evaluation_pipeline = EvaluationPipeline(model, n_folds)

    for dataset_name, info in datasets.items():
        print(f"## running dataset `{dataset_name}`")

        result = evaluation_pipeline.run(info["X"], info["y"])
        evaluation_results[model_name][dataset_name] = result

        print(f"results obtained:")
        result.describe()


# running model `baseline_binary_relevance_model`
## running dataset `scene`
results obtained:
Accuracy: 0.5268 ± 0.13
Hamming Loss: -0.1020 ± 0.03
## running dataset `emotions`
results obtained:
Accuracy: 0.0135 ± 0.01
Hamming Loss: -0.3033 ± 0.02
## running dataset `birds`
results obtained:
Accuracy: 0.4636 ± 0.05
Hamming Loss: -0.0534 ± 0.00
# running model `basic_stacking_model`
## running dataset `scene`


In [18]:
evaluation_results["baseline_binary_relevance_model"]["scene"].raw()

{'fit_time': array([2.0240283 , 1.66600084]),
 'score_time': array([1.19197035, 1.17500257]),
 'test_accuracy': array([0.60880399, 0.58686617]),
 'train_accuracy': array([0.719867  , 0.71345515]),
 'test_hamming_loss': array([-0.08388704, -0.08464949]),
 'train_hamming_loss': array([-0.05444722, -0.05647841])}